speech to text

In [ ]:
#We’ll initially need to install some additional libraries:
!pip install SpeechRecognition
!pip install ffmpeg-python

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install pyaudio

In [ ]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

In [ ]:
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
audio, sr = get_audio()

In [107]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import scipy
scipy.io.wavfile.write('recording.wav', sr, audio)

In [ ]:
#import library
import speech_recognition as sr
#Initiаlize  reсоgnizer  сlаss  (fоr  reсоgnizing  the  sрeeсh)
r = sr.Recognizer()
# Reading Audio file as source
#  listening  the  аudiо  file  аnd  stоre  in  аudiо_text  vаriаble
with sr.AudioFile('recording.wav') as source:
    audio_text = r.listen(source)
# recoginize_() method will throw a request error if the API is unreachable, hence using exception handling
    try:
        # using google speech recognition
        text = r.recognize_google(audio_text)
        print('Converting audio transcripts into text ...')
        print(text)
    except:
         print('Sorry.. run again...')


Converting audio transcripts into text ...
hello hello


text to speech

In [74]:
!pip install gTTS
#Manipulate audio with an simple and easy high level interface.
! pip install pydub 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [75]:
from gtts import gTTS
import IPython #IPython provides a rich architecture for interactive computing
tts=gTTS('Hello, Ishu How are you? . I am hear to solve your problem ')
tts.save('demo.wav')
sound_file='demo.wav'
IPython.display.Audio(sound_file, autoplay=True)

In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install pyngrok

In [ ]:
!ngrok authtoken 2CT4L1Qz5Kjm1ApbwHixPZkJ9cv_5pCAaY2RemGBmeLEvc2yc

In [ ]:
!pip install streamlit==1.1.0

In [ ]:
!nohup streamlit run  app.py &

In [156]:
%%writefile app.py
import streamlit as st 
from PIL import Image
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from flask import Flask ,render_template , request
import nltk
from nltk.chat.util import Chat
from flask_ngrok import run_with_ngrok
from gtts import gTTS
import IPython

qa_pairs = [    [ 'what is your owner name' ,                               ['ishu'] ]  ,
                [ '(.*)name' ,                                              [ 'ishu kumar' ] ]  ,           
                [ 'what is your favourate colour' ,                         ['black'] ]  ,
                [ 'what is your age'              ,                         [ '12' ] ]                     ,
                [ 'what is your favourate book'    ,                        ['Java'] ]        ,
                [ 'what is your favourate food' ,                           [ 'chiken' ] ]      ,                                      
                [ 'what is your creater' ,                                  [ 'ishu kumar' ] ]       ,       
                [ 'what is the favourate colour of your owner' ,            ['black'] ]    ,            
                [ '(hi|HI|Hi|hey|HEY|Hey|HELLO|Hello|hello)',               [' \t hello 👋 \n how can i help u'  ,  '👋 '] ] ,            
                [ '(.*)(location|city|address|place|Place) ?',              ['khagaria bihar']   ]   ,
                [ '(.*)contact(.*)' ,                                       ['call - xxx for more information ℹ '] ]   ,
                [  '(.*)weather(.*)' ,                                      ['it cool 😎 ']    ]
            
           ]

cb = Chat(qa_pairs)


tts=gTTS('Hello, Ishu How are you? . I am hear to solve your problem ')
tts.save('demo.wav')
sound_file='demo.wav'
IPython.display.Audio(sound_file, autoplay=True)


def predict_note_authentication(UserID):

  
  response= cb.respond(UserID)

 
  tts=gTTS(response)
  tts.save('demo.wav')
  sound_file='demo.wav'
  IPython.display.Audio(sound_file, autoplay=True)
  audio_file = open("demo.wav", "rb")
  st.audio(audio_file.read())
  



  
  return response


def main():
    
    html_temp = """
   <div class="" style="background-color:blue;" >
   <div class="clearfix">           
   <div class="col-md-12">
   <center><p style="font-size:40px;color:white;margin-top:10px;">Chat bot</p></center> 
   </div>
   </div>
   </div>
   """
    st.markdown(html_temp,unsafe_allow_html=True)
    
    st.header("Item Purchase Prediction")
    UserID = st.text_input("UserID","")

    resul=""

    if st.button("Predict"):
      result=predict_note_authentication(UserID)
      st.success(' {}'.format(result))

    if st.button("About"):
      st.subheader("Developed by Ishu kumar")
      st.subheader(" Department of Computer Engineering")

if __name__=='__main__':
  main()
   


Overwriting app.py


In [ ]:
from pyngrok import ngrok
url=ngrok.connect(port='8050')
url

In [ ]:
!streamlit run --server.port 80 app.py